# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация

Дата выдачи: 02.02.2024

Мягкий дедлайн: 23:59MSK 19.02.2024

Жесткий дедлайн: 23:59MSK 25.02.2024

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [3]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(y_train.shape[0], -1)
x_test = x_test_pics.reshape(y_test.shape[0], -1)

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        raise NotImplementedError
        return self

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        raise NotImplementedError
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        # Your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        raise NotImplementedError

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.
        new_dim, int: PCA output size.
        use_PCA, bool: whether to include PCA preprocessing.
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        
        if self.classifier == 'svm':
            self.clf = SVC(probability=True)
        elif self.classifier == 'logreg':
            self.clf = LogisticRegression(max_iter=10000)
        else:
            raise ValueError("Invalid classifier. Please choose between 'svm' and 'logreg'.")

    def fit(self, X, y):
        """
        Fit all parts of the pipeline (PCA, RFF, Classification) to the training set.
        """
        if self.use_PCA:
            self.pca = PCA(n_components=self.new_dim)
            X_transformed = self.pca.fit_transform(X)
        else:
            X_transformed = X
        
        self.rff = RBFSampler(n_components=self.n_features, random_state=42)
        X_features = self.rff.fit_transform(X_transformed)
        
        self.clf.fit(X_features, y)
        return self

    def predict_proba(self, X):
        """
        Apply the pipeline to obtain class probabilities for input data.
        """
        if self.use_PCA:
            X_transformed = self.pca.transform(X)
        else:
            X_transformed = X
        
        X_features = self.rff.transform(X_transformed)
        return self.clf.predict_proba(X_features)

    def predict(self, X):
        """
        Apply the pipeline to obtain discrete predictions for input data.
        """
        if self.use_PCA:
            X_transformed = self.pca.transform(X)
        else:
            X_transformed = X
        
        X_features = self.rff.transform(X_transformed)
        return self.clf.predict(X_features)

In [5]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_openml
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline


In [7]:
rff_pipe = RFFPipeline()
rff_pipe.fit(x_train, y_train)
y_pred = rff_pipe.predict(x_test)
y_pred_proba = rff_pipe.predict_proba(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on validation set: {:.2f}%".format(accuracy * 100))

print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy on validation set: 9.93%
Classification Report:
              precision    recall  f1-score   support

           0       0.11      0.10      0.11      1000
           1       0.09      0.09      0.09      1000
           2       0.08      0.08      0.08      1000
           3       0.11      0.12      0.11      1000
           4       0.10      0.10      0.10      1000
           5       0.10      0.10      0.10      1000
           6       0.11      0.11      0.11      1000
           7       0.11      0.10      0.10      1000
           8       0.10      0.09      0.09      1000
           9       0.10      0.10      0.10      1000

    accuracy                           0.10     10000
   macro avg       0.10      0.10      0.10     10000
weighted avg       0.10      0.10      0.10     10000



__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 1 балл)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 6. (Максимум 1 балл)__

Реализуйте класс ядровой Ridge регрессии (Лекция 13, $\S 1.2$), для оптимизации используте градиентный спуск, а не аналитическую формулу. Также подумайте о том, как в формулах правильно учесть свободный коэффициент. Затем адаптируйте вашу реализацию RFF под задачу регрессии. Сравните вашу ядровую регрессию и RFF на синтетических данных.

Функция потерь: 
$$
Q(w) = \frac{1}{2} ||\Phi \Phi^T w - y||^2 + \frac{\lambda}{2} w^T \Phi \Phi^T w \rightarrow \min_w,
$$
где $\Phi \Phi^T = K$, $K = (k(x_i, x_j))_{i, j = 1}^{\ell}$.

Предсказание: 
$
y(x) = k(x)^T w,
$
где $k(x)$ — вектор функций ядра от пар объектов $(x, x_i)_{i=1}^{\ell}$.

Вы можете изменять представленный ниже шаблон по своему усмотрению.

In [ ]:
import numpy as np
from sklearn.base import RegressorMixin
from sklearn.gaussian_process.kernels import RBF

class KernelRidgeRegression(RegressorMixin):
    """
    Kernel Ridge regression class
    """

    def __init__(self,         
        lr=0.01,
        regularization=1.,
        tolerance=1e-2,
        max_iter=1000,
        batch_size=64,
        kernel_scale=1.
    ):
        """
        :param lr: learning rate
        :param regularization: regularization coefficient
        :param tolerance: stopping criterion for square of euclidean norm of weight difference
        :param max_iter: stopping criterion for iterations
        :param batch_size: size of the batches used in gradient descent steps
        :parame kernel_scale: length scale in RBF kernel formula
        """

        self.lr: float = lr
        self.regularization: float = regularization
        self.w: np.ndarray | None = None

        self.tolerance: float = tolerance
        self.max_iter: int = max_iter
        self.batch_size: int = batch_size
        self.loss_history: list[float] = []
        self.kernel = RBF(kernel_scale)

    def calc_loss(self, x: np.ndarray, y: np.ndarray) -> float:
        """
        Calculating loss for x and y dataset
        :param x: features array
        :param y: targets array
        """
        raise NotImplementedError

    def calc_grad(self, x: np.ndarray, y: np.ndarray) -> float:
        """
        Calculating gradient for x and y dataset
        :param x: features array
        :param y: targets array
        """
        raise NotImplementedError

    def fit(self, x: np.ndarray, y: np.ndarray) -> "KernelRidgeRegression":
        """
        Fitting weights for x and y dataset
        :param x: features array
        :param y: targets array
        :return: self
        """
        raise NotImplementedError

    def predict(self, x: np.ndarray) -> np.ndarray:
        """
        Predicting targets for x dataset
        :param x: features array
        :return: prediction: np.ndarray
        """
        raise NotImplementedError

In [22]:
def isOneEditDistance(s: str, t: str) -> bool:
    if len(s) < len(t):
        return isOneEditDistance(t, s)
    len_s, len_t = len(s), len(t)
    if len_s - len_t > 1:
        return False
    for idx in range(len_t):
        if s[idx] != t[idx]:
            if len_s == len_t:
                return s[idx + 1:] == t[idx + 1:]
            else:
                return s[idx + 1:] == t[idx:]
    return len_s == len_t + 1

In [70]:
# Открываем файл для чтения
with open('/Users/eliseishinkarev/MOP_enjoyer/собесы/task_2/universities.txt', 'r') as file:
    # Читаем содержимое файла
    file_contents = file.readlines()

    # Создаем пустое множество
    elements_set = set()

    # Перебираем строки файла и добавляем элементы в множество
    for line in file_contents:
        elements_set.update(line.strip().split('\n'))

# Выводим полученное множество
print(elements_set)

{'Сретенская духовная семинария Русской Православной Церкви', 'Байкальский государственный университет', 'Южно-Уральский государственный институт искусств имени П.И. Чайковского', 'Приволжский исследовательский медицинский университет', 'Дагестанская государственная медицинская академия', 'Нижегородская государственная сельскохозяйственная академия', 'Самарский государственный экономический университет', 'Нижегородский государственный инженерно-экономический университет (Княгининский университет)', 'Дальневосточный государственный медицинский университет', 'Российская таможенная академия', 'Государственный университет морского и речного флота имени адмирала С. О. Макарова', 'Саратовский государственный технический университет им. Гагарина Ю.А.', 'Гуманитарный университет в Екатеринбурге', 'Российский государственный профессионально-педагогический университет', 'Смоленский государственный университет', 'Ступинский филиал Московского авиационного института (национального исследовательско

In [71]:
# Открываем файл для чтения
with open('/Users/eliseishinkarev/MOP_enjoyer/собесы/task_2/queries.txt', 'r') as file:
    # Читаем содержимое файла
    file_contents = file.readlines()

    # Создаем пустое множество
    elements_lst = list()

    # Перебираем строки файла и добавляем элементы в множество
    for line in file_contents:
        elements_lst.append(line.strip())

# Выводим полученное множество
print(elements_lst[:10])

['Северо-западный Государственнмй Медицинский Университет Имени И. И. Мечникова', 'ННоорсосийский яПттигорского годарсэвенного унивеситета', 'Ростовский Институт Дащт Прехпринниматея', 'сАНКТ-пЕтербУГГсКиЙ ГОСУДАРСТВЕННЫЙ АКАДЕМИЧЕСКИЙ ИНСТиТУт живописИ, СКУЛЬПТУЫР И АРХИТЕкТуРЫ ИМЕНИ И.Е. РЕПИНА', 'Санкт-Петербургская духовная академия', 'Тихоокеанскийй гсударственный медицинский уыивеерситгт', 'Пеерский нционшльный иисследвательский политехнический унивреситет', 'Казанский Национальный Исследовательский Технически йУниверситет Им Антуполева-аи', 'РОССИЙСКИЙ ГОУДАРСТВЕННЫЙ  АГРАРНЫЙ УНИВЕРСИТЕТ - МСХА МЕНИК .А. ТМИИРЯЗЕВА', 'Дзержинский Филиал Нижегородскгоо Государственного Университета Имени Ни Лобачевского']


In [72]:
print(len(elements_set), len(elements_lst))

757 50000


In [73]:
! pip install levenshtein

In [74]:
from Levenshtein import distance

In [75]:
lst_ans_uni = []  # Levenshtein.distance("foo", "foobar")

for el in elements_lst:
    min_found_dist = 1000000
    # lst_of_mins = []
    ans = ''

    for st in elements_set:
        if distance(el, st) < min_found_dist:
            # lst_of_mins = [st]
            ans = st
            min_found_dist = distance(el, st)
    lst_ans_uni.append(ans)
    # if min_found_dist == 3:
        # lst_ans_sec.append(f"{el} 3+")
    # if min_found_dist == 2:
        # s = "{} 2 {}".format(el, *lst_of_mins)
        # lst_ans_sec.append(s)
    # if min_found_dist == 1:
        # s = "{} 1 {}".format(el, *lst_of_mins)
        # lst_ans_sec.append(s)
    # if min_found_dist == 0:
        # lst_ans_sec.append(f"{el} 0")

        

In [76]:
print(lst_ans_uni[:15])

['Северо-Западный государственный медицинский университет им. И. И. Мечникова', 'Новороссийский филиал Пятигорского государственного университета', 'Ростовский институт защиты предпринимателя', 'Санкт-Петербургский государственный академический институт живописи, скульптуры и архитектуры имени И. Е. Репина', 'Санкт-Петербургская юридическая академия', 'Тихоокеанский государственный медицинский университет', 'Пермский национальный исследовательский политехнический университет', 'Казанский национальный исследовательский технический университет им. А.Н.Туполева-КАИ', 'Российский государственный аграрный университет - МСХА имени К.А. Тимирязева', 'Дзержинский филиал Нижегородского государственного университета им. Н.И. Лобачевского', 'Уральский федеральный университет имени первого Президента России Б. Н. Ельцина', 'Рязанский государственный агротехнологический университет им. П.А. Костычева', 'Бурятский институт инфокоммуникаций (филиал СибГУТИ)', 'Российская академия музыки имени Гнесины

In [77]:

# Открываем файл для записи
with open('answer_uni.txt', 'w') as file:
    # Записываем каждый элемент массива в отдельной строке файла
    for element in lst_ans_uni:
        file.write(str(element) + '\n')

In [65]:
distance('z', 'Z')

1

In [66]:
# Открываем файл для чтения
with open('/Users/eliseishinkarev/MOP_enjoyer/собесы/task_2/universities.txt', 'r') as file:
    # Читаем содержимое файла
    file_contents = file.readlines()

    # Создаем пустое множество
    elements_set = set()

    # Перебираем строки файла и добавляем элементы в множество
    for line in file_contents:
        elements_set.update(line.strip().split('\n'))

# Выводим полученное множество
print(elements_set)

{'Сретенская духовная семинария Русской Православной Церкви', 'Байкальский государственный университет', 'Южно-Уральский государственный институт искусств имени П.И. Чайковского', 'Приволжский исследовательский медицинский университет', 'Дагестанская государственная медицинская академия', 'Нижегородская государственная сельскохозяйственная академия', 'Самарский государственный экономический университет', 'Нижегородский государственный инженерно-экономический университет (Княгининский университет)', 'Дальневосточный государственный медицинский университет', 'Российская таможенная академия', 'Государственный университет морского и речного флота имени адмирала С. О. Макарова', 'Саратовский государственный технический университет им. Гагарина Ю.А.', 'Гуманитарный университет в Екатеринбурге', 'Российский государственный профессионально-педагогический университет', 'Смоленский государственный университет', 'Ступинский филиал Московского авиационного института (национального исследовательско

In [67]:
new_ans = []  # Levenshtein.distance("foo", "foobar")

for el in lst_ans_uni:
    min_found_dist = 1000000
    # lst_of_mins = []
    ans = ''

    for st in elements_set:
        if distance(el, st) < min_found_dist:
            # lst_of_mins = [st]
            ans = st
            min_found_dist = distance(el, st)
    new_ans.append(ans)

In [68]:
print(new_ans[:15])

['Северо-Западный государственный медицинский университет им. И. И. Мечникова', 'Новороссийский филиал Пятигорского государственного университета', 'Ростовский институт защиты предпринимателя', 'Санкт-Петербургский государственный академический институт живописи, скульптуры и архитектуры имени И. Е. Репина', 'Санкт-Петербургская юридическая академия', 'Тихоокеанский государственный медицинский университет', 'Пермский национальный исследовательский политехнический университет', 'Казанский национальный исследовательский технический университет им. А.Н.Туполева-КАИ', 'Российский государственный аграрный университет - МСХА имени К.А. Тимирязева', 'Дзержинский филиал Нижегородского государственного университета им. Н.И. Лобачевского', 'Уральский федеральный университет имени первого Президента России Б. Н. Ельцина', 'Рязанский государственный агротехнологический университет им. П.А. Костычева', 'Бурятский институт инфокоммуникаций (филиал СибГУТИ)', 'Российская академия музыки имени Гнесины

In [69]:

# Открываем файл для записи
with open('answer_uni2.txt', 'w') as file:
    # Записываем каждый элемент массива в отдельной строке файла
    for element in new_ans:
        file.write(str(element) + '\n')